In [2]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np

In [3]:
weather_data = ['ATL','CLT','DEN','DFW','EWR','IAH','JFK','LAS','LAX','MCO','MIA','ORD','PHX','SEA','SFO']
years = ["2016","2017"]
weather = ['windspeedKmph','winddirDegree','weatherCode','precipMM','visibility','pressure','cloudcover','DewPointF','WindGustKmph','tempF','WindChillF','humidity','date','time', 'airport']

weatherdf = pd.DataFrame()

for airport in weather_data:
    for year in years:
        for i in range(1,13):
            with open("weather/"+airport+"/"+year+"-"+str(i)+".json", 'r') as f:
                data = json.load(f)
            a = json_normalize(data['data']['weather'], 'hourly', ['date'] )
            b = json_normalize(data['data']['request'])
            a['airport'] = str(b['query'])[5:8]
            a = a.filter(weather)
            weatherdf = pd.concat([weatherdf,a])


In [4]:
flightdf = pd.DataFrame()

for year in years:
    for i in range(1,13):
        df = pd.read_csv('Flight-Data/'+year+'_original/'+year+'/On_Time_On_Time_Performance_'+year+'_'+str(i)+'/On_Time_On_Time_Performance_'+year+'_'+str(i)+'.csv', engine = 'python')
        flight = ['FlightDate','Quarter','Year','Month','DayofMonth','DepTime','DepDel15','CRSDepTime','DepDelayMinutes','OriginAirportID','DestAirportID','ArrTime','CRSArrTime','ArrDel15','ArrDelayMinutes','Origin','Dest']
        df = df.filter(flight)
        
        df = df[df['Origin'].isin(weather_data)]
        df = df[df['Dest'].isin(weather_data)]
        
        flightdf = pd.concat([flightdf,df])
        

In [5]:
def alterdeptime(r):
    if(r['CRSDepTime']%100>30):
        return ((r['CRSDepTime']//100 + 1)*100) 
    else:
        return ((r['CRSDepTime']//100)*100)
    
def alterarrtime(r):
    if(r['CRSArrTime']%100>30):
        return ((r['CRSArrTime']//100 + 1)*100) 
    else:
        return ((r['CRSArrTime']//100)*100)

flightdf["newarrtime"] = flightdf.apply(lambda r: alterarrtime(r), axis = 1)
flightdf["newdepttime"] = flightdf.apply(lambda r: alterdeptime(r), axis = 1)


In [ ]:
def unique(r):
  return r["airport"]+"||"+r["date"]+"||"+str(r["time"])

weatherdf["key"] = weatherdf.apply(lambda r: unique(r), axis = 1)


In [20]:
weatherdf

,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,date,time,airport,key
0,11,318,176,0.1,10,1023,100,47,17,49,46,91,2016-01-01,0,ATL,ATL||2016-01-01||0
1,13,317,176,0.0,10,1023,100,44,22,46,42,92,2016-01-01,100,ATL,ATL||2016-01-01||100
2,14,315,122,0.0,10,1023,100,41,26,43,38,92,2016-01-01,200,ATL,ATL||2016-01-01||200
3,16,314,122,0.0,10,1023,100,38,30,40,33,93,2016-01-01,300,ATL,ATL||2016-01-01||300
4,17,314,122,0.0,10,1023,100,38,30,40,33,93,2016-01-01,400,ATL,ATL||2016-01-01||400
5,17,315,122,0.0,10,1024,100,38,29,40,33,93,2016-01-01,500,ATL,ATL||2016-01-01||500
6,18,315,122,0.0,10,1024,100,38,28,40,32,93,2016-01-01,600,ATL,ATL||2016-01-01||600
7,18,317,122,0.0,10,1024,100,38,26,41,33,93,2016-01-01,700,ATL,ATL||2016-01-01||700
8,17,318,122,0.0,10,1025,100,38,23,42,34,92,2016-01-01,800,ATL,ATL||2016-01-01||800
9,17,319,122,0.0,10,1025,100,39,20,42,35,92,2016-01-01,900,ATL,ATL||2016-01-01||900


In [ ]:
def unique(r):
  return r["Origin"]+"||"+r["FlightDate"]+"||"+str(r["newarrtime"])

flightdf["key"] = flightdf.apply(lambda r: unique(r), axis = 1)

final = pd.merge(flightdf, weatherdf, on='key', how='inner')


In [21]:
flightdf

,FlightDate,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,ArrTime,CRSArrTime,ArrDel15,ArrDelayMinutes,Origin,Dest,newarrtime,newdepttime,key
57,2016-01-01,1,2016,1,1,741.0,0.0,745,0.0,14747,12478,1610.0,1602,0.0,8.0,SEA,JFK,1600,800,SEA||2016-01-01||1600
58,2016-01-02,1,2016,1,2,737.0,0.0,745,0.0,14747,12478,1613.0,1602,0.0,11.0,SEA,JFK,1600,800,SEA||2016-01-02||1600
59,2016-01-03,1,2016,1,3,743.0,0.0,745,0.0,14747,12478,1547.0,1602,0.0,0.0,SEA,JFK,1600,800,SEA||2016-01-03||1600
60,2016-01-04,1,2016,1,4,737.0,0.0,745,0.0,14747,12478,1551.0,1602,0.0,0.0,SEA,JFK,1600,800,SEA||2016-01-04||1600
61,2016-01-05,1,2016,1,5,708.0,0.0,710,0.0,14747,12478,1524.0,1527,0.0,0.0,SEA,JFK,1500,700,SEA||2016-01-05||1500
62,2016-01-06,1,2016,1,6,703.0,0.0,710,0.0,14747,12478,1518.0,1527,0.0,0.0,SEA,JFK,1500,700,SEA||2016-01-06||1500
63,2016-01-07,1,2016,1,7,704.0,0.0,710,0.0,14747,12478,1522.0,1527,0.0,0.0,SEA,JFK,1500,700,SEA||2016-01-07||1500
64,2016-01-08,1,2016,1,8,701.0,0.0,710,0.0,14747,12478,1527.0,1527,0.0,0.0,SEA,JFK,1500,700,SEA||2016-01-08||1500
65,2016-01-09,1,2016,1,9,703.0,0.0,710,0.0,14747,12478,1536.0,1527,0.0,9.0,SEA,JFK,1500,700,SEA||2016-01-09||1500
66,2016-01-10,1,2016,1,10,711.0,0.0,710,1.0,14747,12478,1611.0,1527,1.0,44.0,SEA,JFK,1500,700,SEA||2016-01-10||1500


In [ ]:
def unique(r):
  return r["Origin"]+"||"+r["FlightDate"]+"||"+str(r["newdepttime"])

flightdf["key"] = flightdf.apply(lambda r: unique(r), axis = 1)

final = pd.merge(final, weatherdf, on='key', how='inner')


In [16]:
final

,FlightDate,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,...,pressure_y,cloudcover_y,DewPointF_y,WindGustKmph_y,tempF_y,WindChillF_y,humidity_y,date_y,time_y,airport_y
0,2016-01-01,1,2016,1,1,741.0,0.0,745,0.0,14747,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
1,2016-01-01,1,2016,1,1,835.0,0.0,830,5.0,14747,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
2,2016-01-01,1,2016,1,1,821.0,0.0,820,1.0,14747,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
3,2016-01-01,1,2016,1,1,754.0,0.0,759,0.0,14747,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
4,2016-01-01,1,2016,1,1,740.0,0.0,738,2.0,14747,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
5,2016-01-01,1,2016,1,1,742.0,0.0,745,0.0,14747,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
6,2016-01-01,1,2016,1,1,820.0,1.0,800,20.0,14747,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
7,2016-01-01,1,2016,1,1,802.0,0.0,800,2.0,14747,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
8,2016-01-01,1,2016,1,1,919.0,1.0,830,49.0,14747,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
9,2016-01-02,1,2016,1,2,737.0,0.0,745,0.0,14747,...,1024,0,25,8,37,33,63,2016-01-02,800,SEA


In [18]:

del final["key"]
del final["OriginAirportID"]
del final['DestAirportID']
del final['newarrtime']
del final['newdepttime']

final.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1859675 entries, 0 to 1859674
Data columns (total 45 columns):
FlightDate         object
Quarter            int64
Year               int64
Month              int64
DayofMonth         int64
DepTime            float64
DepDel15           float64
CRSDepTime         int64
DepDelayMinutes    float64
ArrTime            float64
CRSArrTime         int64
ArrDel15           float64
ArrDelayMinutes    float64
Origin             object
Dest               object
windspeedKmph_x    object
winddirDegree_x    object
weatherCode_x      object
precipMM_x         object
visibility_x       object
pressure_x         object
cloudcover_x       object
DewPointF_x        object
WindGustKmph_x     object
tempF_x            object
WindChillF_x       object
humidity_x         object
date_x             object
time_x             object
airport_x          object
windspeedKmph_y    object
winddirDegree_y    object
weatherCode_y      object
precipMM_y         object
vis

In [22]:
final.isna().any()

FlightDate         False
Quarter            False
Year               False
Month              False
DayofMonth         False
DepTime             True
DepDel15            True
CRSDepTime         False
DepDelayMinutes     True
ArrTime             True
CRSArrTime         False
ArrDel15            True
ArrDelayMinutes     True
Origin             False
Dest               False
windspeedKmph_x    False
winddirDegree_x    False
weatherCode_x      False
precipMM_x         False
visibility_x       False
pressure_x         False
cloudcover_x       False
DewPointF_x        False
WindGustKmph_x     False
tempF_x            False
WindChillF_x       False
humidity_x         False
date_x             False
time_x             False
airport_x          False
windspeedKmph_y    False
winddirDegree_y    False
weatherCode_y      False
precipMM_y         False
visibility_y       False
pressure_y         False
cloudcover_y       False
DewPointF_y        False
WindGustKmph_y     False
tempF_y            False


In [15]:
final.dropna()

,FlightDate,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,...,pressure_y,cloudcover_y,DewPointF_y,WindGustKmph_y,tempF_y,WindChillF_y,humidity_y,date_y,time_y,airport_y
0,2016-01-01,1,2016,1,1,741.0,0.0,745,0.0,14747,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
1,2016-01-01,1,2016,1,1,835.0,0.0,830,5.0,14747,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
2,2016-01-01,1,2016,1,1,821.0,0.0,820,1.0,14747,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
3,2016-01-01,1,2016,1,1,754.0,0.0,759,0.0,14747,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
4,2016-01-01,1,2016,1,1,740.0,0.0,738,2.0,14747,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
5,2016-01-01,1,2016,1,1,742.0,0.0,745,0.0,14747,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
6,2016-01-01,1,2016,1,1,820.0,1.0,800,20.0,14747,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
7,2016-01-01,1,2016,1,1,802.0,0.0,800,2.0,14747,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
8,2016-01-01,1,2016,1,1,919.0,1.0,830,49.0,14747,...,1030,0,23,8,34,29,65,2016-01-01,800,SEA
9,2016-01-02,1,2016,1,2,737.0,0.0,745,0.0,14747,...,1024,0,25,8,37,33,63,2016-01-02,800,SEA
